# Импортируем необходимые библиотеки

In [1]:
import json
import time
import requests
import yaml
import sql_handler
import postgresql_handler
import clickhouse_handler
import docker_service
from datetime import datetime, timezone




# Опередляем объём импорта и необходимые объекты импорта

In [3]:
# with open('config.yaml', 'r') as file:
#     config = yaml.safe_load(file)

# database_import_mapper = {
#     'MySQL': sql_handler,
#     'PostgreSQL': postgresql_handler,
#     'ClickHouse': clickhouse_handler
# }
# database_type = config.get('database_type')
# print(database_type)
# if database_type in database_import_mapper:
#     handler = database_import_mapper[database_type]
# else:
#     print('Unsupported database type')
    
# handler.test_connection(**config['db'])




In [2]:
import warnings
has_errors = False

# MARK: MAPPERS
table_settings_mapper = {
    'deal': 'deal_fields',
    'contact': 'contact_fields',
    'company': 'company_fields',
    'lead': 'lead_fields',
}

database_import_mapper = {
    'MySQL': sql_handler,
    'PostgreSQL': postgresql_handler,
    'ClickHouse': clickhouse_handler
}

obligatory_fields = ['ID', 'DATE_CREATE']

# MARK: LAMBDAS
def void_to_nonnull(value):
    if not value:
        return ''
    else:
        return value
    
def string_to_decimal(value):
    if not value:
        return 0.0
    else:
        return float(value)
    
def string_to_datetime(value):
    if not value:
        return 0
    else:
        dt = datetime.strptime(value, '%Y-%m-%dT%H:%M:%S%z')
        return int(dt.replace(tzinfo=timezone.utc).timestamp())

# CONFIGURE
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
preprocessing_config = {}
list_of_errors = []
database_type = config.get('database_type')

if database_type in database_import_mapper:
    handler = database_import_mapper[database_type]
else:
    print('Unsupported database type')
    # docker_service.close()

# MARK: CHECK CONNECTION
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
handler.test_connection(**config['db'])

# MARK: CHECK ERRORS
for table_type in config['table_names'].keys():
    print(table_type)
    if config['table_names'][table_type]:
        preprocessing_config[table_type] = {}

        # MergeTree check for ClickHouse
        e,d = handler.check_tabletype_errors(config['table_names'][table_type], **config['db'])
        if e:
            list_of_errors.append(d)

        
        columns = handler.get_columns_and_types(
            config['table_names'][table_type], **config['db'])
        dict_of_columns = {column['name']: (
            column['type'], column['nullable']) for column in columns}
        for obligatory_field in obligatory_fields:
            if obligatory_field not in config[table_settings_mapper[table_type]].keys():
                if has_errors == False:
                    has_errors = True
                list_of_errors.append(
                    f"Column {obligatory_field} not found in config.yaml in {table_settings_mapper[table_type]} settings. \
It is obligatory field.")
        # set_of_columns = set([column['name'] for column in columns])
        for column in config[table_settings_mapper[table_type]]:
            if config[table_settings_mapper[table_type]][column] not in dict_of_columns.keys():
                if has_errors == False:
                    has_errors = True
                list_of_errors.append(
                    f"Column {column} not found in table {table_settings_mapper[table_type]}. Check settings.")
            else:
                nullable = dict_of_columns[config[table_settings_mapper[table_type]][column]][1]
                target_data_type = dict_of_columns[config[table_settings_mapper[table_type]][column]][0]
                current_preproc = preprocessing_config[table_type].get(
                    column, [])

                # TODO Подумать о других кейсах препроцессинга.
                if nullable == False:
                    current_preproc.append(void_to_nonnull)
                if ("Decimal" in target_data_type):
                    current_preproc.append(string_to_decimal)
                if ("DateTime" in target_data_type):
                    current_preproc.append(string_to_datetime)
                if current_preproc:
                    preprocessing_config[table_type][column] = current_preproc


if has_errors:
    print(*list_of_errors, sep='\n')
    print('Please, check your config.yaml file and database. After that, restart the container.')
    # docker_service.close()

# print(preprocessing_config)


Successfully connected to the database.
deal
lead
company
contact


## Выгружаем данные из битрикс24

In [4]:
from tqdm import tqdm

basic_params = {}
if config['filter_date']['lower']:
    basic_params['filter[>=DATE_CREATE]'] = config['filter_date']['lower']
if config['filter_date']['upper']:
    basic_params['filter[<=DATE_CREATE]'] = config['filter_date']['upper']
URL = config['b24_key']
result = {}

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
for table_type in preprocessing_config.keys():
    print(f'{table_type.capitalize()} import started')
    method = f'crm.{table_type}.list.json'
    params = basic_params.copy()
    for num, param in enumerate(config[table_settings_mapper[table_type]]):
        params[f'select[{num}]'] = param
    # print(params)
    r = requests.get(URL + method, params=params).json()
    if 'result' in r.keys():
        result[table_type] = r['result'].copy()
        k = 1
        total = r['total']
        progress_bar = tqdm(total=total, position=0, leave=True)
        time.sleep(0.5)
        while 'next' in r.keys():
            k += 1
            params['start'] = r['next']
            r = requests.get(URL + method, params=params).json()
            result[table_type] += r['result']
            progress_bar.update(50)
    progress_bar.close()

Deal import started


 97%|█████████▋| 900/930 [00:12<00:00, 72.01it/s]


## Выгружаем получившися массивы в базу

In [5]:
# TODO попробовать работать с пакетными запросами

for table_type in result.keys():
    
    for row in result[table_type]:
        for column in preprocessing_config[table_type].keys():
            for func in preprocessing_config[table_type][column]:
                row[column] = func(row[column])

    handler.load_data_to_sql(result[table_type],
                            config['table_names'][table_type], 
                            config[f'{table_type}_fields'], 
                            **config['db'])

  0%|          | 0/930 [00:00<?, ?it/s]

Значения для вставки в таблицу:
[['28720', 'Заявка с сайта metallik.ru. Форма “Стать дилером”', 1689427622, 0.0, 1690651157, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28725', 'Муж купил в Дубне.это близко и его все устроило', 1689468324, 0.0, 1690651157, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28727', 'Форма "Запросить стоимость" на сайте mir-shtaketnika.ru', 1689493794, 77000.0, 1690651158, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28734', 'переходник с коаксиального 60 диаметра ', 1689496582, 19000.0, 1690651158, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28735', 'kratas.ru: Заказать расчёт О2023019421', 1689498799, 11181.4, 1690651159, 1]]
Названия столбцов:
['ID'

 11%|█         | 100/930 [00:46<06:27,  2.14it/s]

Значения для вставки в таблицу:
[['28864', '79035803322 - Входящий звонок', 1689610177, 27000.0, 1690651203, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28865', 'РР 32 0,50 двухсторонний', 1689610257, 0.0, 1690651203, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28866', '79157677318 - Входящий звонок', 1689610344, 0.0, 1690651204, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28867', 'Вячеслав парапеты и профлист', 1689611014, 0.0, 1690651204, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['28868', 'Форма "Заказать расчёт" на сайте profil-mpk.ru', 1689613632, 0.0, 1690651205, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки

 22%|██▏       | 200/930 [01:32<05:37,  2.16it/s]

...

 86%|████████▌ | 800/930 [06:10<01:00,  2.16it/s]

Значения для вставки в таблицу:
[['29622', '79169046691 - Входящий звонок', 1690464506, 25677.9, 1690651527, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29623', 'солманово', 1690465023, 0.0, 1690651527, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29624', '79771428495 - Входящий звонок', 1690465741, 91417.7, 1690651528, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29625', '79166741863 - Входящий звонок', 1690465814, 0.0, 1690651528, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29626', '79262285148 - Входящий звонок', 1690465907, 0.0, 1690651529, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29627', '796

 97%|█████████▋| 900/930 [06:57<00:13,  2.15it/s]

Значения для вставки в таблицу:
[['29722', '79684045018 - Входящий звонок', 1690559940, 0.0, 1690651574, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29723', 'Лист оцинкованный гладкий 0.5х1500х3000 мм RAL 7005 - 2 шт', 1690560161, 0.0, 1690651574, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29724', '74952303086 - Входящий звонок', 1690561640, 660.0, 1690651574, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29725', '74957900000 - Входящий звонок', 1690561951, 0.0, 1690651575, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблицу:
[['29726', 'спам', 1690562019, 0.0, 1690651575, 1]]
Названия столбцов:
['ID', 'TITLE', 'DATE_CREATE', 'OPPORTUNITY', 'version', 'sign']
Значения для вставки в таблиц

 97%|█████████▋| 900/930 [07:11<00:14,  2.09it/s]


## Полезное

In [14]:
columns = handler.get_columns_and_types_sql(
config['table_names']['deal'], **config['db'])
columns

[{'name': 'ID',
  'type': INTEGER(display_width=10, unsigned=True),
  'default': None,
  'comment': None,
  'nullable': False,
  'autoincrement': False},
 {'name': 'TITLE',
  'type': VARCHAR(length=256),
  'default': None,
  'comment': None,
  'nullable': False},
 {'name': 'TYPE_ID',
  'type': VARCHAR(length=20),
  'default': None,
  'comment': None,
  'nullable': True},
 {'name': 'STAGE_ID',
  'type': VARCHAR(length=50),
  'default': None,
  'comment': None,
  'nullable': False},
 {'name': 'PROBABILITY',
  'type': VARCHAR(length=20),
  'default': None,
  'comment': None,
  'nullable': True},
 {'name': 'OPPORTUNITY',
  'type': DECIMAL(precision=20, scale=2),
  'default': None,
  'comment': None,
  'nullable': False},
 {'name': 'LEAD_ID',
  'type': VARCHAR(length=20),
  'default': None,
  'comment': None,
  'nullable': True},
 {'name': 'COMPANY_ID',
  'type': VARCHAR(length=20),
  'default': None,
  'comment': None,
  'nullable': True},
 {'name': 'CONTACT_ID',
  'type': VARCHAR(length=2

In [8]:
type(columns[0]['type'])

sqlalchemy.dialects.mysql.types.INTEGER

In [12]:
import sqlalchemy.dialects.mysql as mysql_dialect

types = dir(mysql_dialect.types)
print(*types, sep='\n')

BIGINT
BIT
CHAR
DATETIME
DECIMAL
DOUBLE
FLOAT
INTEGER
LONGBLOB
LONGTEXT
MEDIUMBLOB
MEDIUMINT
MEDIUMTEXT
NCHAR
NUMERIC
NVARCHAR
REAL
SMALLINT
TEXT
TIME
TIMESTAMP
TINYBLOB
TINYINT
TINYTEXT
VARCHAR
YEAR
_FloatType
_IntegerType
_MatchType
_NumericType
_StringType
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__spec__
datetime
exc
sqltypes
util


In [24]:
a = []
b = [1,2]
c = [3,4]
a += b
a += c
a

[1, 2, 3, 4]

In [71]:
import sqlalchemy.dialects.postgresql as postgresql_dialect
import sqlalchemy_clickhouse.base as clickhouse_dialect

types_pg = dir(postgresql_dialect)
types_ch = dir(clickhouse_dialect)

print("PostgreSQL types:")
print(*types_pg, sep='\n')

print("\nClickHouse types:")
print(*types_ch, sep='\n')

PostgreSQL types:
ARRAY
All
Any
BIGINT
BIT
BOOLEAN
BYTEA
CHAR
CIDR
CreateEnumType
DATE
DATERANGE
DOUBLE_PRECISION
DropEnumType
ENUM
ExcludeConstraint
FLOAT
HSTORE
INET
INT4RANGE
INT8RANGE
INTEGER
INTERVAL
Insert
JSON
JSONB
MACADDR
MONEY
NUMERIC
NUMRANGE
OID
REAL
REGCLASS
SMALLINT
TEXT
TIME
TIMESTAMP
TSRANGE
TSTZRANGE
TSVECTOR
UUID
VARCHAR
__all__
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
aggregate_order_by
array
array_agg
asyncpg
base
compat
dialect
dml
ext
hstore
insert
json
pg8000
psycopg2
psycopg2cffi
pygresql
pypostgresql
ranges

ClickHouse types:
ARRAY
BIGINT
BINARY
BOOLEAN
CHAR
ClickHouseCompiler
ClickHouseDialect
ClickHouseExecutionContext
ClickHouseIdentifierPreparer
ClickHouseTypeCompiler
DATE
DATETIME
DECIMAL
FLOAT
INTEGER
PGCompiler
PGIdentifierPreparer
REAL
SMALLINT
TIME
TIMESTAMP
VARCHAR
VERSION
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__spec__
colspecs
compiler
default
dialect
expression
isch

In [70]:
import sys
!{sys.executable} -m pip install sqlalchemy_clickhouse

     |████████████████████████████████| 43 kB 117 kB/s eta 0:00:011
  Created wheel for sqlalchemy-clickhouse: filename=sqlalchemy_clickhouse-0.1.5.post0-py3-none-any.whl size=18658 sha256=063d5b73c0a98e88b00f3cbdb85c79512e05f86cbac8436b9ee8baaaeb6d0edd
  Stored in directory: /Users/master/Library/Caches/pip/wheels/f9/8a/7e/b68fa72db2160e97c3338ff739577ac18fe8b6e9361f8d8dc6
Successfully built sqlalchemy-clickhouse
